# MAX-CUT for graphs

In [12]:
import numpy as np
import scipy as sp
import math
import pulp
from random import *
from pyvis.network import Network as netviz
import networkx as net

### Generate a G(n,p)-random graph

In [58]:
N,p = 100, 0.05
G = net.gnp_random_graph(N,p)
print(len(G.nodes))
print(len(G.edges))

100
258


## Create the adjacency matrix of G
The function `adjacency_matrix()` returns a sparse matrix (i.e. only the coordinates of non-null entries). We turn it into a dense matrix before we can use it with the function `todense()`.

In [49]:
A = net.adjacency_matrix(G).todense()

## Define a function to calculate the cut number for given `x`

In [50]:
def CUT(G,x):
    A = net.adjacency_matrix(G).todense()
    n = len(G.nodes)
    cut = 0
    if len(x) != n:
        print("ERROR length of x does not match dimension of G")
        return(cut)
    else:
        for i in range(1,n):
            for j in range(1,n):
                cut += 1/4*A.A[i][j]*(1-x[i]*x[j])
        return(cut)  

## Generate $X \sim \mathcal{U} (\{-1,1\})$ and calculate CUT(G,X)

In [59]:
def rand_part(G):
    "return a random partition of V(G) represented by a vector with independent symmetric Bernoulli entries"
    X = []
    for i in range(N):
        X.append(1-2*round(random()))
    return(X)
X = rand_part(G)
CUT(G,X)

115.0

## Define a (brute-force) function to calculate MAXCUT
*WARNING* the runtime is $O(2^n)$ and _will_ take long for Graph sizes above $N=15$

In [52]:
def iterate(x):
    x[0] = x[0]*(-1)
    for i in range(len(x)):
        if (x[i] == 1 ):
            try:
                x[i+1] = x[i+1]*(-1)
            except IndexError:
                break
        else:
            break
    return(x)
    
    
def MAXCUT(G):
    n = len(G.nodes)
    x = [1 for i in range(n)]
    x0 = x
    cut = 0
    h = 0
    for i in range (2**n):
        h = CUT(G,x)
        if h > cut:
            cut = h
            x0 = x
            print(f'found a better CUT. NEW CUT# = {cut}')
        x = iterate(x)
    return(cut,x0)
        
    
    

## A $(0.5-\varepsilon)$-approximation algorithm for MAXCUT
Let $\varepsilon>0$ we start with a random partition of $V$, represented by a vector $X$ with independent symmetric Bernoulli entries. Then flip the entries of $X$ one by one and check if the cut number of the new cutset `CUT(G,X)` has improved. If so, update $X$ and continue until `CUT(G,X)` $\geq (0.5-\varepsilon)|E|$. As long as there are vertices where less than half of the incident edges are cut, the cut number is improved by flipping such a vertex. Hence, the algorithm improves can find a vertex to flip as long as `CUT(G,X)` $< 0.5 |E|$. Furthermore, the cut number increases by at least 1 with every flip, so the the number of flips is upper bounded by $|E|$.

**Exercise** Implement this algorithm!

In [53]:
def maxcut_approx(G):
    ...
    return(X)


## Draw the partitioned Graph

In [65]:
def draw_part_graph(G,x):
    n = len(G.nodes)
    if len(x) != n:
        print("ERROR length of x does not match dimension of G")
        return
    for v in G.nodes():
        if x[v] == 1:
            G.nodes[v]["color"] = "red"
        else:
            G.nodes[v]["color"] = "blue"
    edges = [e for e in G.edges]
    for e in edges:
        if x[e[0]]*x[e[1]] == -1:
            G.edges[e]["color"] = "red"
        else:
            G.edges[e]["color"] = "black"
    viz = netviz(height=800, width=800, notebook=True)
    #viz.barnes_hut()
    #viz.repulsion()
    #viz.hrepulsion()
    viz.force_atlas_2based()
    viz.from_nx(G)
    viz.show("net.html")    
    return(viz)

viz = draw_part_graph(G,X)
viz.show("net.html")


## Calculate the MAXCUT-norm of G

In [111]:
cut, x0 = MAXCUT(G)
print(cut)

found a better CUT. NEW CUT# = 4.0
found a better CUT. NEW CUT# = 6.0
found a better CUT. NEW CUT# = 7.0
found a better CUT. NEW CUT# = 8.0
found a better CUT. NEW CUT# = 9.0
found a better CUT. NEW CUT# = 10.0
found a better CUT. NEW CUT# = 11.0
found a better CUT. NEW CUT# = 12.0
found a better CUT. NEW CUT# = 13.0
found a better CUT. NEW CUT# = 14.0
found a better CUT. NEW CUT# = 15.0
found a better CUT. NEW CUT# = 16.0
16.0
